In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName('Cleaning') \
    .master('local[*]') \
    .getOrCreate()

In [135]:
from pyspark.sql.functions import *
#from pyspark.sql import DataFrame, SQLContext

In [77]:
input_path = '/home/jovyan/work/data/'
users_path = f'{input_path}all_users_v1/'
comments_path = f'{input_path}all_comments_v1/'
posts_path = f'{input_path}all_posts_v1/'

In [319]:
users = spark.read.csv(users_path, multiLine=True, header=True, inferSchema=True) # multiline - allows \n to appear in a single row
comments = spark.read.csv(comments_path, multiLine=True, header=True, inferSchema=True)
posts = spark.read.csv(posts_path, multiLine=True, header=True, inferSchema=True)

In [264]:
print(f'Comments-rows: {comments.count()}')
print(f'Users-rows: {users.count()}')
print(f'Posts-rows: {posts.count()}')

Comments-rows: 140785
Users-rows: 147577
Posts-rows: 91936


In [320]:
users.printSchema()

root
 |-- _AboutMe: string (nullable = true)
 |-- _AccountId: integer (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _DisplayName: string (nullable = true)
 |-- _Id: integer (nullable = true)
 |-- _LastAccessDate: timestamp (nullable = true)
 |-- _Location: string (nullable = true)
 |-- forum: string (nullable = true)



In [258]:
comments.printSchema()

root
 |-- _CreationDate: timestamp (nullable = true)
 |-- _Id: integer (nullable = true)
 |-- _PostId: integer (nullable = true)
 |-- _Text: string (nullable = true)
 |-- _UserDisplayName: string (nullable = true)
 |-- _UserId: integer (nullable = true)
 |-- forum: string (nullable = true)
 |-- _AccountId: integer (nullable = true)



In [159]:
comments_count=comments.groupBy('_AccountId', 'forum').agg(count('*').cast(DoubleType()).alias('comments_number'))

In [164]:
comments_count.show(5)

+----------+------+---------------+
|_AccountId| forum|comments_number|
+----------+------+---------------+
|     73848|french|            1.0|
|   2355716|french|           15.0|
|  14230173|french|            1.0|
|   5042025|french|            1.0|
|     29567|french|            7.0|
+----------+------+---------------+
only showing top 5 rows



In [226]:
# PostTypeId
# - 1: Question
# - 2: Answer
post_cols = ['_AnswerCount', '_Body', '_CommentCount', '_PostTypeId', '_Tags', '_Title', '_ViewCount',
             'forum', 'LastEditorAccountId', 'OwnerAccountId']
answer_cols = ['_CommentCount', '_Body', 'forum', 'LastEditorAccountId']
question_cols = ['_AnswerCount', '_Body', '_CommentCount', '_Tags', '_Title', '_ViewCount', 
                 'forum', 'LastEditorAccountId', 'OwnerAccountId']

In [227]:
post_answers = posts.filter(posts['_PostTypeId'] == 2)
post_answers = post_answers[answer_cols]
post_answers = post_answers.filter(post_answers['LastEditorAccountId'] > 0)
post_questions = posts.filter(posts['_PostTypeId'] == 1)
post_questions = post_questions[question_cols]

In [228]:
print(post_answers.count())
print(post_questions.count())

45416
43727


In [310]:
answers = post_answers.groupBy('LastEditorAccountId', 'forum') \
                      .agg(count('*').cast(DoubleType()).alias('edits_of_answers')#, 
                          # sum(post_answers['_CommentCount']).cast(DoubleType()).alias('comments_to_answer')
                          )
answers.show(5)

+-------------------+---------+----------------+
|LastEditorAccountId|    forum|edits_of_answers|
+-------------------+---------+----------------+
|            5134905|chemistry|             2.0|
|           10834649|chemistry|             1.0|
|           12861263|chemistry|             1.0|
|            9522198|chemistry|             2.0|
|            7553617|chemistry|             4.0|
+-------------------+---------+----------------+
only showing top 5 rows



In [309]:
editors = post_questions.groupBy('LastEditorAccountId', 'forum') \
                      .agg(count('*').cast(DoubleType()).alias('edits_of_questions')#,
                           #sum(post_questions['_AnswerCount']).cast(DoubleType()).alias('answers_to_edited_question'),
                           #sum(post_questions['_CommentCount']).cast(DoubleType()).alias('comments_to_edited_question'),
                           #sum(post_questions['_ViewCount']).cast(DoubleType()).alias('views_of_edited_question')
                          )
editors.show(5)

+-------------------+---------+-----------------+
|LastEditorAccountId|    forum|edits_of_question|
+-------------------+---------+-----------------+
|            7553617|chemistry|              4.0|
|            1877825|chemistry|              1.0|
|            7745813|chemistry|              4.0|
|            1844444|chemistry|              1.0|
|            2103819|chemistry|              1.0|
+-------------------+---------+-----------------+
only showing top 5 rows



In [301]:
owners = post_questions.groupBy('OwnerAccountId', 'forum') \
                       .agg(count('*').cast(DoubleType()).alias('created_questions')#,
                           # sum(post_questions['_AnswerCount']).cast(DoubleType()).alias('answers_to_question'),
                           # sum(post_questions['_CommentCount']).cast(DoubleType()).alias('comments_to_question'),
                           # sum(post_questions['_ViewCount']).cast(DoubleType()).alias('views_of_question')
                           )
owners.show(5)

+--------------+---------+-----------------+
|OwnerAccountId|    forum|created_questions|
+--------------+---------+-----------------+
|       7157712|chemistry|              1.0|
|      14589975|chemistry|              1.0|
|      16760944|chemistry|              1.0|
|       1877825|chemistry|              1.0|
|       7745813|chemistry|             10.0|
+--------------+---------+-----------------+
only showing top 5 rows



Join Users with comments and posts(editors, owners)

In [321]:
users=users.withColumn('ActivityDays', datediff(users['_LastAccessDate'], users['_CreationDate']).cast(DoubleType()))

Join with comments

In [322]:
users = users.join(comments_count, on=['_AccountId', 'forum'], how='left')
users = users.withColumn('comments_every_day', round(users['ActivityDays'] / users['comments_number'], 1))

users = users.na.fill({'comments_number': 0, 'comments_every_day': 0})

Join with posts (answers)

In [328]:
users = users.withColumnRenamed('forum', 'forum2')
users = users.join(answers, 
           (users['_AccountId'] == answers['LastEditorAccountId']) & (users['forum2'] == answers['forum']), 
            how='left')
users = users.drop('forum', 'LastEditorAccountId')

Join with posts (questions)

In [332]:
users = users.join(editors, 
                   (users['_AccountId'] == editors['LastEditorAccountId']) & (users['forum2'] == editors['forum']), 
                   how='left')
users = users.drop('forum')

users = users.join(owners, 
                   (users['_AccountId'] == owners['OwnerAccountId']) & (users['forum2'] == owners['forum']), 
                   how='left')
users = users.drop('forum', 'LastEditorAccountId', 'OwnerAccountId')
users = users.withColumnRenamed('forum2', 'forum')
#users = users.withColumn('comments_every_day', round(users['ActivityDays'] / users['comments_number'], 1))

Fill NA

In [339]:
users = users.na.fill({'edits_of_answers': 0, 'edits_of_questions': 0, 'created_questions': 0})

Calculate activities of user (edits, comments) per activity days 

In [342]:
users = users.withColumn('activity_answer_edits', round(users['ActivityDays'] / users['edits_of_answers'], 1))
users = users.withColumn('activity_question_edits', round(users['ActivityDays'] / users['edits_of_questions'], 1))
users = users.withColumn('activity_questions', round(users['ActivityDays'] / users['created_questions'], 1))

In [344]:
users = users.na.fill({'activity_answer_edits': 0, 'activity_question_edits': 0, 'activity_questions': 0})

In [343]:
users.printSchema()

root
 |-- _AccountId: integer (nullable = true)
 |-- forum: string (nullable = true)
 |-- _AboutMe: string (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _DisplayName: string (nullable = true)
 |-- _Id: integer (nullable = true)
 |-- _LastAccessDate: timestamp (nullable = true)
 |-- _Location: string (nullable = true)
 |-- ActivityDays: double (nullable = true)
 |-- comments_number: double (nullable = false)
 |-- comments_every_day: double (nullable = false)
 |-- edits_of_answers: double (nullable = false)
 |-- edits_of_question: double (nullable = false)
 |-- created_questions: double (nullable = false)
 |-- activity_answer_edits: double (nullable = true)
 |-- activity_question_edits: double (nullable = true)
 |-- activity_questions: double (nullable = true)



In [359]:
users.filter(isnull(users['_AboutMe'])).count()
# Dużo nulli w opisie AboutMe -> nie biore pod uwage

111610

In [363]:
users.filter(isnull(users['_Location'])).count()

105805

In [374]:
location_split = users.select(split(users['_Location'], ',').alias('location'))

In [345]:
cols = ['_AccountId', 'forum', '_Location', 'ActivityDays', 
        'comments_number', 'edits_of_answers', 'edits_of_question', 'created_questions', 
        'comments_every_day', 'activity_answer_edits', 'activity_question_edits', 'activity_questions']

In [ ]:
users = users[cols]

In [362]:
users.write.csv(path=input_path+'cleaned_data_users', mode='overwrite', header=True) #, sep=',')